Importing the necessary libraries for both linear regression and asynchronous

In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import concurrent.futures

Load the dataset

In [15]:
dataset = pd.read_csv('Position_Salaries.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

Split data into training and testing sets

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Model training and Evaluating  on the test set

In [17]:
def train_polynomial_regression(degree):
    """Trains a Polynomial Regression model with the given degree."""
    poly_features = PolynomialFeatures(degree=degree)
    X_poly_train = poly_features.fit_transform(X_train)
    
    model = LinearRegression()
    model.fit(X_poly_train, y_train)
    
    
    X_poly_test = poly_features.transform(X_test)# Evaluating on the test set
    score = model.score(X_poly_test, y_test)
    
    return degree, model, poly_features, score

Trying different polynomial degree

Using ThreadPollExecutor to train models in parallel

Collecting the result when they became abailable

finding the best model based on there score 

Also predicting the results with best model 

Here i predicted for level 6.5


In [18]:
if __name__ == '__main__':
    degrees = [2, 3, 4, 5]  
    
    # Use a ThreadPoolExecutor 
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(train_polynomial_regression, degree) for degree in degrees]
        
        # Collect the results as they become available
        results = []
        for future in concurrent.futures.as_completed(futures):
            try:
                degree, model, poly_features, score = future.result()
                results.append((degree, model, poly_features, score))
                print(f"Polynomial Regression with degree {degree} trained. Score: {score}")
            except Exception as e:
                print(f"Error training model: {e}")
    
    # finding the best model based on the score
    best_degree, best_model, best_poly_features, best_score = max(results, key=lambda x: x[3])
    print(f"Best degree: {best_degree}, Best score: {best_score}")

    # making predictions of salary  using the best model for level 6.5
    new_level = np.array([[6.5]])
    new_level_poly = best_poly_features.transform(new_level)
    predicted_salary = best_model.predict(new_level_poly)
    print(f"Predicted salary for level 6.5: {predicted_salary[0]:.2f}")

Polynomial Regression with degree 3 trained. Score: 0.8762695647830158
Polynomial Regression with degree 2 trained. Score: 0.7666492889299886
Polynomial Regression with degree 4 trained. Score: 0.9714666803843642
Polynomial Regression with degree 5 trained. Score: 0.9950481764354088
Best degree: 5, Best score: 0.9950481764354088
Predicted salary for level 6.5: 171803.58
